In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# from huggingface_hub import login
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import pipeline

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# login(token = os.getenv('HUGGINGFACEHUB_API_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\acer\.cache\huggingface\token
Login successful


In [4]:
# Open the file in read mode
with open('service_links.txt', 'r') as file:
    # Read all lines from the file
    lines = file.readlines()

# Strip any leading/trailing whitespace from each line and store in a list
links = [line.strip() for line in lines]

# Print the list of links
print(links)

['https://www.coventry.gov.uk/a-to-z/service/87/abandoned-vehicles', 'https://www.coventry.gov.uk/a-to-z/service/349/abuse-adults', 'https://www.coventry.gov.uk/a-to-z/service/847/abuse-children', 'https://www.coventry.gov.uk/a-to-z/service/403/accessibility-website', 'https://www.coventry.gov.uk/a-to-z/service/534/accounts-inspection', 'https://www.coventry.gov.uk/a-to-z/service/876/address-change', 'https://www.coventry.gov.uk/a-to-z/service/49/adoption', 'https://www.coventry.gov.uk/a-to-z/service/54/adult-carers', 'https://www.coventry.gov.uk/a-to-z/service/211/adult-education', 'https://www.coventry.gov.uk/a-to-z/service/951/adult-social-care', 'https://www.coventry.gov.uk/a-to-z/service/89/air-pollution', 'https://www.coventry.gov.uk/a-to-z/service/89/air-quality', 'https://www.coventry.gov.uk/a-to-z/service/421/alcohol-advice-and-support', 'https://www.coventry.gov.uk/a-to-z/service/999/alcohol-misuse-support', 'https://www.coventry.gov.uk/a-to-z/service/821/allesley-parish-coun

In [5]:
coventry_links = tuple(links)
coventry_links

('https://www.coventry.gov.uk/a-to-z/service/87/abandoned-vehicles',
 'https://www.coventry.gov.uk/a-to-z/service/349/abuse-adults',
 'https://www.coventry.gov.uk/a-to-z/service/847/abuse-children',
 'https://www.coventry.gov.uk/a-to-z/service/403/accessibility-website',
 'https://www.coventry.gov.uk/a-to-z/service/534/accounts-inspection',
 'https://www.coventry.gov.uk/a-to-z/service/876/address-change',
 'https://www.coventry.gov.uk/a-to-z/service/49/adoption',
 'https://www.coventry.gov.uk/a-to-z/service/54/adult-carers',
 'https://www.coventry.gov.uk/a-to-z/service/211/adult-education',
 'https://www.coventry.gov.uk/a-to-z/service/951/adult-social-care',
 'https://www.coventry.gov.uk/a-to-z/service/89/air-pollution',
 'https://www.coventry.gov.uk/a-to-z/service/89/air-quality',
 'https://www.coventry.gov.uk/a-to-z/service/421/alcohol-advice-and-support',
 'https://www.coventry.gov.uk/a-to-z/service/999/alcohol-misuse-support',
 'https://www.coventry.gov.uk/a-to-z/service/821/allesl

In [6]:
len(coventry_links)

693

In [10]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create a WebBaseLoader instance to load documents from web sources
loader = WebBaseLoader(
    web_paths=coventry_links,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            name=("h1", "p")
        )
    ),
)
# Load documents from web sources using the loader
documents = loader.load()

# Replace \xa0 with normal spaces in the text content of each document
for doc in documents:
    doc.page_content = doc.page_content.replace('\xa0', ' ').replace('\n', ' ')

# Initialize a RecursiveCharacterTextSplitter for splitting text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# Split the documents into chunks using the text_splitter
docs = text_splitter.split_documents(documents)

In [11]:
len(docs)

708

In [12]:
docs[18]

Document(metadata={'source': 'https://www.coventry.gov.uk/a-to-z/service/89/air-quality'}, page_content='cycle more.Anyone can make a difference by just making small changes to their everyday routines, The Clean Air Hub has everything you need to know about air pollution in one place.Road traffic is a major source of air pollution in Coventry. You can help reduce this impact by:Further advice is available from the Clear Air Hub website. Email: airquality@coventry.gov.uk 2024 © Coventry City Council')

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [14]:
from langchain_milvus import Zilliz

vectorstore = Zilliz.from_documents(
    documents=docs,
    embedding=embeddings,
    connection_args={
        "uri": os.getenv("MILVUS_URI"),
        "token": os.getenv("MILVUS_TOKEN"),
    },
    drop_old=True,
    auto_id=True,
)

In [15]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# from langchain_huggingface import HuggingFaceEndpoint
# from langchain.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain

import gradio as gr

In [16]:
PROMPT_TEMPLATE = """
Human: You are an AI assistant for Coventry, and provides answers to questions by using fact based information when possible.
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ('system', PROMPT_TEMPLATE),
#         ('human', '{input}')
#     ]
# )

In [17]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-70B")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-70B")

In [ ]:
def coventry_response(message, history):
    query = message
    vectorstore.similarity_search(query, k=1)
    
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    
    retriever = vectorstore.as_retriever()
    
    prompt = PromptTemplate(
        template=PROMPT_TEMPLATE, input_variables=["context", "question"]
    )

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    res = rag_chain.invoke(query)

    return res

# print(coventry_response("What to do with a lost dog?", ""))
gr.ChatInterface(coventry_response).launch()